# 03: Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/03_visualization.ipynb)

Visualizing snowpack data is crucial for understanding patterns and communicating results. This notebook shows you how to create effective plots with xsnow data.

## What You'll Learn

- Plotting snow profiles (depth vs properties)
- Creating time series plots
- Multi-panel figures
- Customizing plots for presentations
- Temperature and density profiles


### Learning objectives
- Load xsnow sample data for consistent plotting demos.
- Create layered profile, time-series, and heatmap visualizations.
- Customize Matplotlib styling to highlight snowpack signals.
- Compare multiple locations with shared axes for quick decisions.

**Prerequisites**
- [ ] Notebook 02 selection/filtering knowledge.
- [ ] Familiarity with Matplotlib basics.
- [ ] Ability to interpret snowpack variables like density and temperature.


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:
# Run.
%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow


In [ ]:
# Run.
import xsnow
import matplotlib.pyplot as plt
import numpy as np

print('Loading xsnow sample data for visualization...')
try:
    ds = xsnow.single_profile_timeseries()
    print('✅ xsnow dataset ready for plotting')
except Exception as exc:
    raise RuntimeError('Install xsnow before continuing: pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow') from exc

plt.style.use('default')


**Explain.** A single helper call supplies layered snowpack data so every plot stays grounded in reality.


In [ ]:
# Check for understanding: dataset ready
assert ds is not None
assert 'temperature' in ds.data_vars


## Part 1: Plotting a Single Snow Profile
**Show.** We'll build focused temperature and density plots for one snow profile.


In [ ]:
# Run.
if ds is None:
    raise RuntimeError('Load the dataset above first.')

profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile['z'].values
temp = profile['temperature'].values

fig, ax = plt.subplots(figsize=(6, 8))
ax.plot(temp, depth, color='tomato', linewidth=2, label='Temperature')
ax.axvline(0, color='gray', linestyle='--', alpha=0.3)
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Depth from surface (m)')
ax.set_title('Temperature Profile')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()


**Explain.** A single profile slice keeps axes manageable and reveals the vertical gradient clearly.


In [ ]:
# Check for understanding: profile selection
assert 'layer' in profile.dims
assert depth.shape[0] == profile.sizes['layer']


### Density Profile
**Show.** Reuse the selected profile to visualize density vs. depth.


In [ ]:
# Run.
density = profile['density'].values

fig, ax = plt.subplots(figsize=(6, 8))
ax.plot(density, depth, color='steelblue', linewidth=2, label='Density')
ax.set_xlabel('Density (kg/m³)')
ax.set_ylabel('Depth from surface (m)')
ax.set_title('Density Profile')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()


**Explain.** Using the same depth array aligns structural and thermal patterns for easy comparison.


In [ ]:
# Check for understanding: density array
assert density.shape == depth.shape
assert float(density.max()) > 0


## Part 2: Multi-Panel Profile Plot
**Show.** Display multiple variables side-by-side to compare signatures.


In [ ]:
# Run.
variables = ['temperature', 'density', 'grain_size']
fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)
for ax, var in zip(axes, variables):
    if var in profile.data_vars:
        ax.plot(profile[var].values, depth, linewidth=2)
        ax.set_xlabel(var.replace('_', ' ').title())
        ax.set_title(var.replace('_', ' ').title())
        ax.grid(True, alpha=0.3)
        ax.invert_yaxis()
axes[0].axvline(0, color='gray', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


**Explain.** Shared axes make it obvious when variables co-vary or diverge throughout the profile.


In [ ]:
# Check for understanding: multi-panel setup
assert len(axes) == 3
assert axes[0].get_shared_y_axes().joined(axes[0], axes[1])


## Part 3: Time Series Plots
**Show.** Track changes through time using line charts.


In [ ]:
# Run.
if ds is None:
    raise RuntimeError('Load the dataset above first.')

hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(hs_series['time'].values, hs_series.values, linewidth=2)
ax.set_xlabel('Date')
ax.set_ylabel('Snow Height (m)')
ax.set_title('Snow Height Through Time')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


**Explain.** Tracking height over time surfaces periods of rapid accumulation or melt.


In [ ]:
# Check for understanding: HS series
assert hs_series.dims == ('time',)
assert hs_series.size == ds.sizes['time']


### Surface Temperature Time Series
**Show.** Chart layer-0 temperature to highlight diurnal swings.


In [ ]:
# Run.
surface_temp = ds['temperature'].isel(location=0, slope=0, realization=0, layer=0)
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(surface_temp['time'].values, surface_temp.values, color='crimson', linewidth=2)
ax.axhline(0, color='gray', linestyle='--', alpha=0.4)
ax.set_xlabel('Date')
ax.set_ylabel('Surface Temperature (°C)')
ax.set_title('Surface Temperature Time Series')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


**Explain.** Overlaying the freezing line highlights warm pulses and refreeze events.


In [ ]:
# Check for understanding: surface temp dims
assert surface_temp.dims == ('time',)
assert surface_temp.size == ds.sizes['time']


## Part 4: Heatmap (Depth-Time)
**Show.** Render a depth-time heatmap to see vertical evolution.


In [ ]:
# Run.
if ds is None:
    raise RuntimeError('Load the dataset above first.')

location_slice = ds.isel(location=0, slope=0, realization=0)
temp_data = location_slice['temperature']
layer_depth = -location_slice['z'].isel(time=0).values
plot_array = temp_data.values.T

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.pcolormesh(temp_data['time'].values, layer_depth, plot_array, cmap='RdYlBu_r', shading='auto')
plt.colorbar(im, ax=ax, label='Temperature (°C)')
ax.set_xlabel('Date')
ax.set_ylabel('Depth from surface (m)')
ax.set_title('Temperature Evolution (Depth-Time)')
ax.invert_yaxis()
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


**Explain.** The depth-time heatmap quickly reveals persistent warm or cold layers.


In [ ]:
# Check for understanding: heatmap prep
assert plot_array.shape == (layer_depth.size, temp_data['time'].size)
assert im.get_array().ndim >= 1


## Part 5: Comparing Multiple Locations
**Show.** Overlay site series to spot spatial differences.


In [ ]:
# Run.
n_locations = ds.sizes.get('location', 0)
if n_locations <= 1:
    print('Only one location available; duplicate the dataset to compare sites.')
else:
    times = ds['time'].values
    fig, ax = plt.subplots(figsize=(10, 5))
    for idx_loc in range(n_locations):
        name = ds['location'].values[idx_loc]
        hs_line = ds['HS'].isel(location=idx_loc, slope=0, realization=0)
        ax.plot(times, hs_line.values, linewidth=2, label=f'Location {name}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Snow Height (m)')
    ax.set_title('Snow Height by Location')
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.show()


**Explain.** Shared axes make it easy to spot outliers or lagging accumulation trends.


In [ ]:
# Check for understanding: location count
assert n_locations >= 1
if n_locations > 1:
    assert len(ax.lines) == n_locations


## Part 6: Customizing Plots
**Show.** Polish a profile plot with annotations and styling.


In [ ]:
# Run.
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile['z'].values
temp = profile['temperature'].values

fig, ax = plt.subplots(figsize=(8, 10))
ax.plot(temp, depth, color='firebrick', linewidth=3, marker='o', markersize=5, label='Temperature')
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Freezing')
ax.set_xlabel('Temperature (°C)', fontsize=14, fontweight='bold')
ax.set_ylabel('Depth from surface (m)', fontsize=14, fontweight='bold')
ax.set_title('Styled Snow Temperature Profile', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle=':')
ax.legend(fontsize=12)
ax.text(0.02, 0.95, 'Surface', transform=ax.transAxes, fontsize=10, va='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.6))
ax.invert_yaxis()
plt.tight_layout()
plt.show()


**Explain.** Thoughtful styling keeps attention on the message—here, how temperature changes with depth.


In [ ]:
# Check for understanding: styling choices
assert ax.get_xlabel() == 'Temperature (°C)'
assert ax.get_legend() is not None


### Play
Try new color maps or smoothing windows to highlight different snowpack stories while keeping plots responsive.


In [ ]:
# Run.
smooth = 3  # Try 2–5 to adjust smoothing

series = ds['temperature'].isel(location=0, slope=0, realization=0, layer=0)
smoothed = series.rolling(time=smooth, center=True, min_periods=1).mean()
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(series['time'], series, label='Original', alpha=0.4)
ax.plot(series['time'], smoothed, label=f'Smoothed ({smooth})', color='navy')
ax.set_title('Play with Smoothing')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


## Practice
Work these visualization prompts before opening the answers.


1. Plot density time series for a single layer and annotate notable drops.
2. Build a two-row figure with temperature and density profiles stacked vertically.
3. Export the heatmap figure to PNG with a descriptive filename.


<details>
<summary>Solutions</summary>

1. Select `ds['density'].isel(layer=0)` and use `ax.annotate` at minima.
2. Use `plt.subplots(2, 1, sharex=True)` and reuse the profile arrays for each axis.
3. After creating the heatmap, call `fig.savefig(f'heatmap_{ds.location.values[0]}.png', dpi=150)`.

</details>


## Summary
- Profile, time-series, and heatmap visuals reveal snowpack structure from multiple perspectives.
- Matplotlib styling choices help you communicate safety-critical cues clearly.
- Quick parameter tweaks (colormap, smoothing) adapt plots to new stories.
